In [ ]:
from pyspark.sql import functions as F

# Pull the 
var = dbutils.widgets.getAll()
for key in [*var]:
    print(f"{key}: {var[key]}")

In [ ]:
def process_bronze(checkpoint_path, file_path, full_table_name):
    raw_df = (
        spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option(
            "cloudFiles.schemaLocation", checkpoint_path
        )
        .load(file_path)
    )

    # Cast all columns to string to handle column type mismatch through the years
    df_string = raw_df.select([F.col(c).cast("string") for c in raw_df.columns])

    # write to table incrementally
    query = (
        df_string.writeStream
        .foreachBatch(row_count)
        .option(
            "checkpointLocation", checkpoint_path
        )
        .option("mergeSchema", "true")
        .outputMode("append")
        .trigger(availableNow=True)  # this means incremental batch
        .toTable(full_table_name)
    )
    query.awaitTermination()

In [ ]:
process_bronze(
    var["checkpoint_path"],
    var["file_path"],
    var["full_table_name"]
)